<a href="https://colab.research.google.com/github/pablo-francisco/Exploratory-Data-Analysis-of-E-commerce-Brazil/blob/main/olist_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importações de bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import datetime
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif,SelectPercentile
from imblearn.under_sampling import RandomUnderSampler
from tqdm import tqdm

# Coleta e organização dos dados

In [ ]:
df_customers = pd.read_csv('olist_customers_dataset.csv')
df_location = pd.read_csv('olist_geolocation_dataset.csv')
df_order_item = pd.read_csv('olist_order_items_dataset.csv')
df_order_payments = pd.read_csv('olist_order_payments_dataset.csv')
df_reviews = pd.read_csv('olist_order_reviews_dataset.csv')
df_orders = pd.read_csv('olist_orders_dataset.csv')
df_products = pd.read_csv('olist_products_dataset.csv')
df_sellers = pd.read_csv('olist_sellers_dataset.csv')

Junção de dados de acordo com as colunas chave, customer_id e order_id.

In [ ]:
dados_relacao_consumer = [df_customers[['customer_id','customer_city','customer_state']],
df_products.drop(['product_name_lenght','product_description_lenght','product_photos_qty'],axis=1) ]

dados_relacao_order = [
                df_order_item[['order_id','product_id','shipping_limit_date','price','freight_value']],
                df_order_payments[['order_id','payment_type','payment_installments','payment_value']],
                df_reviews[['order_id','review_score']],
                df_orders[['order_id','customer_id','order_status','order_purchase_timestamp','order_delivered_customer_date','order_estimated_delivery_date']]

                ]


In [ ]:
def ultimate_merge(dados,dados_1):
  n_data = dados[0]
  for i in dados[1:]:

    n_data = pd.merge(n_data, i, on='order_id', how='inner')

  n_data = pd.merge(n_data, dados_1[0], on='customer_id', how='inner')
  n_data = pd.merge(n_data, dados_1[1], on='product_id', how='inner')
  return n_data


In [ ]:
df = ultimate_merge(dados_relacao_consumer,dados_relacao_order)
df= df.drop(['shipping_limit_date'],axis=1)
df

,order_id,product_id,price,freight_value,payment_type,payment_installments,payment_value,review_score,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,customer_city,customer_state,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.9,13.29,credit_card,2,72.19,5,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-20 23:43:48,2017-09-29 00:00:00,campos dos goytacazes,RJ,cool_stuff,650.0,28.0,9.0,14.0
1,130898c0987d1801452a8ed92a670612,4244733e06e7ecb4970a6e2683c13e61,55.9,17.96,boleto,1,73.86,5,e6eecc5a77de221464d1c4eaff0a9b64,delivered,2017-06-28 11:52:20,2017-07-13 20:39:29,2017-07-26 00:00:00,jatai,GO,cool_stuff,650.0,28.0,9.0,14.0
2,532ed5e14e24ae1f0d735b91524b98b9,4244733e06e7ecb4970a6e2683c13e61,64.9,18.33,credit_card,2,83.23,4,4ef55bf80f711b372afebcb7c715344a,delivered,2018-05-18 10:25:53,2018-06-04 18:34:26,2018-06-07 00:00:00,belo horizonte,MG,cool_stuff,650.0,28.0,9.0,14.0
3,6f8c31653edb8c83e1a739408b5ff750,4244733e06e7ecb4970a6e2683c13e61,58.9,16.17,credit_card,3,75.07,5,30407a72ad8b3f4df4d15369126b20c9,delivered,2017-08-01 18:38:42,2017-08-09 21:26:33,2017-08-25 00:00:00,sao jose dos pinhais,PR,cool_stuff,650.0,28.0,9.0,14.0
4,7d19f4ef4d04461989632411b7e588b9,4244733e06e7ecb4970a6e2683c13e61,58.9,13.29,credit_card,4,72.19,5,91a792fef70ecd8cc69d3c7feb3d12da,delivered,2017-08-10 21:48:40,2017-08-24 20:04:21,2017-09-01 00:00:00,conselheiro lafaiete,MG,cool_stuff,650.0,28.0,9.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117324,ffebd80e3291e811c308365936897efd,4cc4d02efc8f249c13355147fb44e34d,129.9,51.20,credit_card,1,181.10,5,050309b91cc5e04e68841938e7984aaf,delivered,2018-07-13 20:04:05,2018-07-23 19:44:45,2018-08-07 00:00:00,conceicao de macabu,RJ,ferramentas_jardim,6700.0,35.0,12.0,22.0
117325,ffee31fb4b5e35c9123608015637c495,b10ecf8e33aaaea419a9fa860ea80fb5,99.0,13.52,credit_card,2,112.52,4,11e0f43ab4e2d2c48348dd9332c0ef80,delivered,2018-08-18 10:00:59,2018-08-21 12:18:57,2018-08-23 00:00:00,sao jose dos pinhais,PR,moveis_decoracao,2300.0,37.0,30.0,20.0
117326,fff7c4452f050315db1b3f24d9df5fcd,dd469c03ad67e201bc2179ef077dcd48,736.0,20.91,credit_card,1,756.91,5,dec8952e97ef6124259c56914fb3569c,delivered,2017-06-01 16:53:03,2017-06-08 13:04:40,2017-06-26 00:00:00,volta redonda,RJ,relogios_presentes,400.0,19.0,9.0,15.0
117327,fffa82886406ccf10c7b4e35c4ff2788,bbe7651fef80287a816ead73f065fc4b,229.9,44.02,credit_card,7,273.92,4,a5201e1a6d71a8d21e869151bd5b4085,delivered,2017-12-18 16:33:07,2018-01-08 18:23:10,2018-01-24 00:00:00,tenente ananias,RN,esporte_lazer,2700.0,60.0,15.0,15.0


# Análise inicial dos dados

Identificando tipo de dados de cada coluna assim como a proporção de valores nulos relevantes

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117329 entries, 0 to 117328
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       117329 non-null  object 
 1   product_id                     117329 non-null  object 
 2   price                          117329 non-null  float64
 3   freight_value                  117329 non-null  float64
 4   payment_type                   117329 non-null  object 
 5   payment_installments           117329 non-null  int64  
 6   payment_value                  117329 non-null  float64
 7   review_score                   117329 non-null  int64  
 8   customer_id                    117329 non-null  object 
 9   order_status                   117329 non-null  object 
 10  order_purchase_timestamp       117329 non-null  object 
 11  order_delivered_customer_date  114858 non-null  object 
 12  order_estimated_delivery_date 

Organizando dados de datas de string para datetime

In [ ]:
colunas_datas = ['order_purchase_timestamp','order_delivered_customer_date','order_estimated_delivery_date']
for k in colunas_datas:
  df[k] = pd.to_datetime(df[k])


Criação de novas features


*  Dias_para_entrega: Tempo estimado para a entrega do produto.
*   Dias_atraso: Valores negativos indicam que foi entregue com X dias de antecedêcia, e positivos com atraso de X dias.
*   qntd_produtos_cesta: Quantidade de itens que o consumidor comprará naquela transação.

*   qntd_compras: Quantas compras já foram feitas para aquele produto, indicando sua popularidade entre os consumidores.




In [ ]:
df['Dias_para_entrega'] = (df['order_purchase_timestamp'] - df['order_estimated_delivery_date']).dt.days
df['Dias_atraso'] = (df['order_delivered_customer_date'].dt.date - df['order_estimated_delivery_date'].dt.date).dt.days
mapeamento_qntd_compras = df.groupby(['order_id']).size().to_dict()
mapeamento_qntd_produtos = df.groupby(['product_id']).size().to_dict()
df['qntd_produtos_cesta'] = df['order_id'].map(mapeamento_qntd_compras)
df['qntd_compras'] = df['product_id'].map(mapeamento_qntd_produtos)

In [ ]:
df

,order_id,product_id,price,freight_value,payment_type,payment_installments,payment_value,review_score,customer_id,order_status,...,customer_state,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,Dias_para_entrega,qntd_produtos_cesta,qntd_compras,Dias_atraso
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.9,13.29,credit_card,2,72.19,5,3ce436f183e68e07877b285a838db11a,delivered,...,RJ,cool_stuff,650.0,28.0,9.0,14.0,-16,1,11,-9.0
1,130898c0987d1801452a8ed92a670612,4244733e06e7ecb4970a6e2683c13e61,55.9,17.96,boleto,1,73.86,5,e6eecc5a77de221464d1c4eaff0a9b64,delivered,...,GO,cool_stuff,650.0,28.0,9.0,14.0,-28,1,11,-13.0
2,532ed5e14e24ae1f0d735b91524b98b9,4244733e06e7ecb4970a6e2683c13e61,64.9,18.33,credit_card,2,83.23,4,4ef55bf80f711b372afebcb7c715344a,delivered,...,MG,cool_stuff,650.0,28.0,9.0,14.0,-20,1,11,-3.0
3,6f8c31653edb8c83e1a739408b5ff750,4244733e06e7ecb4970a6e2683c13e61,58.9,16.17,credit_card,3,75.07,5,30407a72ad8b3f4df4d15369126b20c9,delivered,...,PR,cool_stuff,650.0,28.0,9.0,14.0,-24,1,11,-16.0
4,7d19f4ef4d04461989632411b7e588b9,4244733e06e7ecb4970a6e2683c13e61,58.9,13.29,credit_card,4,72.19,5,91a792fef70ecd8cc69d3c7feb3d12da,delivered,...,MG,cool_stuff,650.0,28.0,9.0,14.0,-22,1,11,-8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117324,ffebd80e3291e811c308365936897efd,4cc4d02efc8f249c13355147fb44e34d,129.9,51.20,credit_card,1,181.10,5,050309b91cc5e04e68841938e7984aaf,delivered,...,RJ,ferramentas_jardim,6700.0,35.0,12.0,22.0,-25,1,1,-15.0
117325,ffee31fb4b5e35c9123608015637c495,b10ecf8e33aaaea419a9fa860ea80fb5,99.0,13.52,credit_card,2,112.52,4,11e0f43ab4e2d2c48348dd9332c0ef80,delivered,...,PR,moveis_decoracao,2300.0,37.0,30.0,20.0,-5,1,1,-2.0
117326,fff7c4452f050315db1b3f24d9df5fcd,dd469c03ad67e201bc2179ef077dcd48,736.0,20.91,credit_card,1,756.91,5,dec8952e97ef6124259c56914fb3569c,delivered,...,RJ,relogios_presentes,400.0,19.0,9.0,15.0,-25,1,1,-18.0
117327,fffa82886406ccf10c7b4e35c4ff2788,bbe7651fef80287a816ead73f065fc4b,229.9,44.02,credit_card,7,273.92,4,a5201e1a6d71a8d21e869151bd5b4085,delivered,...,RN,esporte_lazer,2700.0,60.0,15.0,15.0,-37,1,1,-16.0


#Análise de dados

Foram identificados atrasos nos prazos de entregas e suas quantidades, utilizando métricas arbritárias de classificação dadas como:



*   Atraso Leve (Atrasos de até 7 dias);
*   Atraso Médio (Atrasos de 7 a 14 dias);
*   Atraso Grave (Atrasos de 14 a 30 dias);
*   Atraso Gravíssimo (Atrasos maiores que 30 dias).



In [ ]:
def classificar_atraso(dados):
  if dados >30:
    x = 'Gravíssimo'
  elif dados>=14:
    x = 'Grave'
  elif dados>=7:
    x = 'Médio'
  else :
    x= 'Leve'

  return x


In [ ]:
atrasados = df[df['Dias_atraso'] >0]


atrasados['Class_Atraso']= atrasados['Dias_atraso'].map(classificar_atraso)
atrasados

<ipython-input-75-0873b43a1eab>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atrasados['Class_Atraso']= atrasados['Dias_atraso'].map(classificar_atraso)


,order_id,product_id,price,freight_value,payment_type,payment_installments,payment_value,review_score,customer_id,order_status,...,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,Dias_para_entrega,qntd_produtos_cesta,qntd_compras,Dias_atraso,Class_Atraso
5,a0f9acf0b6294ed8561e32cde1a966bc,4244733e06e7ecb4970a6e2683c13e61,55.90,26.93,credit_card,1,82.83,5,4851b3bca4a48b395dfad7aa57c3f72a,delivered,...,cool_stuff,650.0,28.0,9.0,14.0,-28,1,11,7.0,Médio
18,07e2a9d486a8394f2700230396e8d5f0,ac6c3623068f30de03045865e4e10089,209.90,61.93,credit_card,1,271.83,1,0d0a49286a8610cff0b0ca481ac5d14b,delivered,...,ferramentas_jardim,3750.0,35.0,40.0,30.0,-34,1,12,10.0,Médio
72,7753606fb1d0067848f8a1a7ae5abefa,d63c1011f49d98b976c352955b1c4bea,59.99,8.88,credit_card,1,137.74,4,5c4a6f6b98ec2d780b10f1cc88ef208f,delivered,...,beleza_saude,950.0,35.0,25.0,20.0,-14,2,58,7.0,Médio
73,7753606fb1d0067848f8a1a7ae5abefa,d63c1011f49d98b976c352955b1c4bea,59.99,8.88,credit_card,1,137.74,4,5c4a6f6b98ec2d780b10f1cc88ef208f,delivered,...,beleza_saude,950.0,35.0,25.0,20.0,-14,2,58,7.0,Médio
75,8bec10779c439cbca16b5d86fae19f72,d63c1011f49d98b976c352955b1c4bea,59.99,9.34,credit_card,2,69.33,1,1fa8883eaf19c9adbe1ca73b326f9a34,delivered,...,beleza_saude,950.0,35.0,25.0,20.0,-8,1,58,3.0,Leve
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117265,ff3a6dc71d198630c018885d4049ef6b,925e74f05a815d63be377531002e29b3,48.90,11.85,credit_card,1,60.75,4,e749ef65bf52337f19fbf21c4b61c53e,delivered,...,cama_mesa_banho,1500.0,36.0,14.0,36.0,-19,1,1,2.0,Leve
117267,ff3f2d5b004f8051849318f201add7c9,69bfbc73c85e0c111060d84260d5fd64,249.99,49.61,credit_card,3,299.60,1,3935a46119fd8d69b602e645cedca908,delivered,...,cool_stuff,10300.0,80.0,40.0,36.0,-17,1,1,13.0,Médio
117296,ffa5e4c604dea4f0a59d19cc2322ac19,59d8034bb2a82cd5092e78d42148fa44,29.99,15.10,credit_card,1,90.18,5,665fe3b7ab435cda0fad0c3cbb1c51dd,delivered,...,pet_shop,250.0,16.0,4.0,11.0,-29,2,1,1.0,Leve
117297,ffa5e4c604dea4f0a59d19cc2322ac19,bd421826916d3e1d445cb860cea3c0fb,29.99,15.10,credit_card,1,90.18,5,665fe3b7ab435cda0fad0c3cbb1c51dd,delivered,...,NaN,250.0,16.0,4.0,11.0,-29,2,1,1.0,Leve


In [ ]:
atrasados.groupby(['Class_Atraso']).size().sort_values(ascending=False).to_frame()

,0
Class_Atraso,
Leve,3617
Médio,2004
Grave,1380
Gravíssimo,364


O peso do produto está correlacionado com o custo de frete?

A correlação de pearson indica que a relação é de 61.13%, logo na maioria dos casos o peso influenciará no frete.


In [ ]:
df[['product_weight_g','freight_value']].corr().iloc[0,1] # NUM-NUM

0.611302285391081

Qual é a taxa de cancelamento de pedidos?

In [ ]:
df_teste = df.groupby(['order_id','order_status']).size().to_frame().reset_index().groupby('order_status').size().to_frame()
df_teste1 = (100*df_teste/df_teste.sum()).reset_index()
df_teste1.columns = ['Porcentagem' if x != 'order_status' else x for x in df_teste1.columns ]
df_teste1['Valores'] = df_teste.values

Taxa de cancelamento de aproximadamente 0.45%.

In [ ]:
df_teste1

,order_status,Porcentagem,Valores
0,approved,0.002043,2
1,canceled,0.453450,444
2,delivered,97.870624,95831
3,invoiced,0.313534,307
4,processing,0.301279,295
5,shipped,1.052943,1031
6,unavailable,0.006128,6


# Pré-processamento

Separando features consideradas importantes por meio de uma opinião pessoal.

In [ ]:
df_modelo=df[['order_id','product_id','price','freight_value','payment_type','review_score','order_status','customer_city','Dias_para_entrega','qntd_produtos_cesta','qntd_compras']]
df_modelo

,order_id,product_id,price,freight_value,payment_type,review_score,order_status,customer_city,Dias_para_entrega,qntd_produtos_cesta,qntd_compras
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.9,13.29,credit_card,5,delivered,campos dos goytacazes,-16,1,11
1,130898c0987d1801452a8ed92a670612,4244733e06e7ecb4970a6e2683c13e61,55.9,17.96,boleto,5,delivered,jatai,-28,1,11
2,532ed5e14e24ae1f0d735b91524b98b9,4244733e06e7ecb4970a6e2683c13e61,64.9,18.33,credit_card,4,delivered,belo horizonte,-20,1,11
3,6f8c31653edb8c83e1a739408b5ff750,4244733e06e7ecb4970a6e2683c13e61,58.9,16.17,credit_card,5,delivered,sao jose dos pinhais,-24,1,11
4,7d19f4ef4d04461989632411b7e588b9,4244733e06e7ecb4970a6e2683c13e61,58.9,13.29,credit_card,5,delivered,conselheiro lafaiete,-22,1,11
...,...,...,...,...,...,...,...,...,...,...,...
117324,ffebd80e3291e811c308365936897efd,4cc4d02efc8f249c13355147fb44e34d,129.9,51.20,credit_card,5,delivered,conceicao de macabu,-25,1,1
117325,ffee31fb4b5e35c9123608015637c495,b10ecf8e33aaaea419a9fa860ea80fb5,99.0,13.52,credit_card,4,delivered,sao jose dos pinhais,-5,1,1
117326,fff7c4452f050315db1b3f24d9df5fcd,dd469c03ad67e201bc2179ef077dcd48,736.0,20.91,credit_card,5,delivered,volta redonda,-25,1,1
117327,fffa82886406ccf10c7b4e35c4ff2788,bbe7651fef80287a816ead73f065fc4b,229.9,44.02,credit_card,4,delivered,tenente ananias,-37,1,1


Transformando os dados para forma numérica, onde a identificação dos dados será mais fácil, por exemplo os id's, e na aplicação de modelos de Machine/Deep Learning

In [ ]:
df_copia = df_modelo.copy()
#Foram considerados os valores cancelados como 1, e quaisquer outros como 0.
df_copia['order_status'] = df_copia['order_status'].apply(lambda x: 0 if x != 'canceled' else 1)
cat_columns = ['order_id','product_id','payment_type','customer_city']
encoders = []
for colunas in cat_columns:
  temp_encoder = LabelEncoder()
  df_copia[colunas] = temp_encoder.fit_transform(df_copia[colunas])
  encoders.append(temp_encoder)
df_copia

,order_id,product_id,price,freight_value,payment_type,review_score,order_status,customer_city,Dias_para_entrega,qntd_produtos_cesta,qntd_compras
0,0,8577,58.9,13.29,1,5,0,738,-16,1,11
1,7297,8577,55.9,17.96,0,5,0,1944,-28,1,11
2,31819,8577,64.9,18.33,1,4,0,452,-20,1,11
3,42653,8577,58.9,16.17,1,5,0,3546,-24,1,11
4,47782,8577,58.9,13.29,1,5,0,1034,-22,1,11
...,...,...,...,...,...,...,...,...,...,...,...
117324,97885,9933,129.9,51.20,1,5,0,1005,-25,1,1
117325,97889,22611,99.0,13.52,1,4,0,3546,-5,1,1
117326,97902,28388,736.0,20.91,1,5,0,4086,-25,1,1
117327,97906,24046,229.9,44.02,1,4,0,3824,-37,1,1


## Feature Scaling

Foi realizada a divisão entre dados de entrada (Dados relevantes) e saída (Objetivo) e realizado um escalonamento entre os dados para haver uma variação menor de valores.

In [ ]:
X = df_copia.drop(['order_status',],axis=1)
y = df_copia['order_status']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0, stratify = y)

In [ ]:
def escalonar(X_train,X_test):
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  X_train_scaled = pd.DataFrame(X_train_scaled)
  X_train_scaled.columns = X_train.columns
  X_test_scaled = pd.DataFrame(X_test_scaled)
  X_test_scaled.columns = X_test.columns
  return X_train_scaled, X_test_scaled, scaler

In [ ]:
X_train_scaled, X_test_scaled,_ = escalonar(X_train,X_test)

## Feature Sampling

Realizado um UnderSampling onde reduzimos a quantidades de dados que são da classe majoritária com o objetivo de deixar os dados igualmente divididos entre cancelados e não cancelados.

In [ ]:
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_scaled, y_train)


# Fatores que mais contribuem para o cancelamento de um pedido.

Foram empregadas técnicas para a seleção de Features que mais importam para a identificação de um possível cancelamento.

## Random Forest - Embedding

Utilizando um modelo de Machine Learning clássico, o Random Forest, para a detecção de relações entre os dados. O modelo pode mudar os resultados a depender dos seus hyperparâmetros.

In [ ]:
def Feature_Selection_EMBEDD(X,y,best_f):
  model = RandomForestClassifier()
  model.fit(X,y)
  importances = model.feature_importances_
  indices = np.argsort(importances)[::-1]
  num_features = best_f
  top_indices = indices[:num_features]
  top_importances = importances[top_indices]
  colunas_importantes = X.columns[top_indices].values
  features_importantes = pd.DataFrame(colunas_importantes,index=top_importances).reset_index()
  features_importantes.columns = ['Importancia','Feature']
  return features_importantes

## ANOVA - Analysis of Variance

Foi utilizado como base os valores F-value e tem dois critérios de escolha:



*   Escolher o número de Features relevantes.
*   Determinar um intervalo de confiança para o p-value, geralmente de 5 %.



In [ ]:
def Feature_Selection_ANOVA(X,y,confidence=False):
  ordenar_melhores = f_classif(X,y)
  anova_feature_selection = pd.DataFrame(ordenar_melhores,index=['F-value','p-value'],columns=X.columns).T.sort_values('p-value')
  if confidence!=False:
    confidence_interval = confidence
    df_temp =  anova_feature_selection[anova_feature_selection['p-value'] <= confidence_interval].sort_values('F-value',ascending=False)
  else:
    df_temp = anova_feature_selection.sort_values('F-value',ascending=False)
  return df_temp

Com isso a comparação entre os métodos de seleção de Features obtiveram resultados similares, o melhor método a ser escolhido pode ser baseado em argumentos evidentes no próprio conjunto de dados ou utilizar as Features identificada por cada técnica e comparar os desempenhos nos modelos de previsão.

In [ ]:
colunas_mais_relacionadas = 5
FS_RANDOM = Feature_Selection_EMBEDD(X_train_resampled, y_train_resampled,colunas_mais_relacionadas)
FS_ANOVA = Feature_Selection_ANOVA(X_train_resampled, y_train_resampled)

features_from_models = pd.concat([FS_RANDOM['Feature'],FS_ANOVA.reset_index().iloc[:colunas_mais_relacionadas]['index']],axis=1)
features_from_models.columns = ['Random_Forest','ANOVA']
features_from_models.index = features_from_models.index + 1
features_from_models

,Random_Forest,ANOVA
1,review_score,review_score
2,qntd_compras,qntd_compras
3,freight_value,price
4,price,Dias_para_entrega
5,order_id,order_id


#Realizando uma identificação de quais produtos foram comprados juntos e com maior frequência.

Organizando os produtos que cada consumidor comprou em conjunto.

In [ ]:
colab_df = df[['order_id','customer_id','product_id','review_score']]
colab_df
label_encoder = LabelEncoder()
label_encoder1 = LabelEncoder()
colab_df['customer_id'] = label_encoder.fit_transform(colab_df['customer_id'])
colab_df['product_id'] = label_encoder1.fit_transform(colab_df['product_id'])
customer_products = colab_df.groupby('customer_id')['product_id'].apply(list).reset_index()
customer_products

<ipython-input-106-74c6ea4c2cda>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colab_df['customer_id'] = label_encoder.fit_transform(colab_df['customer_id'])
<ipython-input-106-74c6ea4c2cda>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colab_df['product_id'] = label_encoder1.fit_transform(colab_df['product_id'])


,customer_id,product_id
0,0,[12871]
1,1,[16979]
2,2,[20220]
3,3,[22348]
4,4,[17255]
...,...,...
97911,97911,[6078]
97912,97912,"[24602, 24602, 24602]"
97913,97913,[27435]
97914,97914,[29476]


Criar uma matriz que correlacione quais produtos foram comprados juntos, e quantas vezes foram comprados.

In [ ]:
def coletar_prods_iguais(df_temp):
  products = df_temp['product_id'].unique()

  matrix = np.zeros((len(products), len(products)), dtype=int)

  # Preencher a matriz com 1 se os produtos foram comprados juntos
  for _, group in df_temp.groupby('customer_id'):
      product_list = group['product_id'].unique()
      for i, product1 in enumerate(product_list):
          for j, product2 in enumerate(product_list):
              if i != j:
                  matrix[np.where(products == product1)[0][0], np.where(products == product2)[0][0]] += 1


  # Criar o DataFrame a partir da matriz
  result_df_temp = pd.DataFrame(matrix, index=products, columns=products)
  return result_df_temp

In [ ]:
matrizes_por_prods = coletar_prods_iguais(colab_df)
matrizes_por_prods

,8577,29455,25545,15243,21967,30694,18089,11060,6350,8960,...,28387,2119,29908,32468,16473,9933,22611,28388,24046,6866
8577,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25545,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9933,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28388,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24046,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


DataFrame com o produto escolhido na coluna e os produtos que foram mais comprados nas linhas, com o N° de compras sendo o valor da tabela.

In [ ]:
produto_escolhido = 19640
num_prox = 5 # Filtra o número de produtos que mais foram comprados juntos

produtos_mais_comprados_juntos = matrizes_por_prods[produto_escolhido][matrizes_por_prods[produto_escolhido].values > 0].sort_values(ascending=False).to_frame().iloc[:num_prox]
produtos_mais_comprados_juntos


,19640
6939,29
31129,6
8063,2
30588,2
654,1
